# Downward recursion relations for $\mathcal{I}_v$ and $\mathcal{J}_v$

#### Validation of the analytical solution to the integrals

Let's import some stuff:

In [10]:
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as pl
from mpmath import ellipe, ellipk, fac2
from sympy import factorial
%matplotlib inline
epsabs=1e-12
epsrel=1e-12

Let's define the numerical form of the integrals:

In [2]:
def I(v, k):
    """Return the integral I, evaluated numerically."""
    kappa = 2 * np.arcsin(k)
    func = lambda x: np.sin(x) ** (2 * v)
    res, err = quad(func, -0.5 * kappa, 0.5 * kappa, epsabs=epsabs, epsrel=epsrel)
    return res

def J(v, k):
    """Return the integral J, evaluated numerically."""
    kappa = 2 * np.arcsin(k)
    func = lambda x: np.sin(x) ** (2 * v) * (1 - k ** (-2) * np.sin(x) ** 2) ** 1.5
    res, err = quad(func, -0.5 * kappa, 0.5 * kappa, epsabs=epsabs, epsrel=epsrel)
    return res

Let's also define our analytical recursion relations:

In [12]:
def ISeries(v, k, nterms=30):
    """Return the integral I, evaluated as a series."""
    res = 0
    for j in range(nterms):
        res += fac2(2 * j - 1) / (2 ** j * factorial(j) * (2 * j + 2 * v + 1)) * k ** (2 * j)
    return 2 * k ** (1 + 2 * v) * float(res)

def JSeries(v, k, nterms=30):
    """Return the integral J, evaluated as a series."""
    res = 0
    for j in range(nterms):
        res += fac2(2 * j - 1) * fac2(2 * j + 2 * v - 1) / \
               (2 ** (2 * j + v) * factorial(j) * factorial(j + v + 2)) * k ** (2 * j)
    return 3 * np.pi / 4 * k ** (1 + 2 * v) * float(res)

def IDownward(v, k, vmax=10, nterms=30):
    """Return the integral I, evaluated recursively."""
    if v == vmax:
        return ISeries(v, k, nterms)
    else:
        return (2 / (2 * v + 1)) * ((v + 1) * IDownward(v + 1, k, vmax, nterms) + k ** (2 * v + 1) * np.sqrt(1 - k ** 2))

def JDownward(v, k, vmax=10, nterms=30):
    """Return the integral J, evaluated recursively."""
    if (v == vmax) or (v == vmax - 1):
        return JSeries(v, k, nterms)
    else:
        return 1 / ((2 * v + 1) * k ** 2) * (2 * (3 + v + (1 + v) * k ** 2) * JDownward(v + 1, k, vmax, nterms) -
                                             (2 * v + 7) * JDownward(v + 2, k, vmax, nterms))

Let's show that the error is close to the machine limit for $k < \frac{1}{2}$ and $v < 10$. (Recall from the text that these expressions are very slow to converge for $k > \frac{1}{2}$, in which case we perform upward recursion instead.)

In [11]:
maxrel = 0
maxfrac = 0
for k in np.logspace(-5, np.log10(0.5), 100):
    for v in range(10):
        I1 = I(v, k)
        I2 = IDownward(v, k)
        rel = np.abs(I1 - I2)
        frac = np.abs(rel / I1)
        if rel > maxrel:
            maxrel = rel
        if frac > maxfrac:
            maxfrac = frac
print("The maximum relative error on I is %.4e." % maxrel)
print("The maximum fractional error on I is %.4e." % maxfrac)

The maximum relative error on I is 1.1102e-16.
The maximum fractional error on I is 2.6404e-15.


In [14]:
maxrel = 0
maxfrac = 0
for k in np.logspace(-5, np.log10(0.5), 100):
    for v in range(10):
        J1 = J(v, k)
        J2 = JDownward(v, k)
        rel = np.abs(J1 - J2)
        frac = np.abs(rel / J1)
        if rel > maxrel:
            maxrel = rel
        if frac > maxfrac:
            maxfrac = frac
print("The maximum relative error on J is %.4e." % maxrel)
print("The maximum fractional error on J is %.4e." % maxfrac)

The maximum relative error on J is 7.0499e-15.
The maximum fractional error on J is 1.0977e-05.


Woot. Note that as before, the fractional error on $\mathcal{J}_v$ is rather large (but still only about 10 ppm) because the magnitude of the value of $\mathcal{J}_v$ can become very small for certain values of $k$ and $v$, in which case the numerical integral is not very accurate!
■